In [11]:
# Read in the data
import pandas as pd
df = pd.read_csv('/content/UConn SJI Data_.csv')
df.head()

,Participant,Timestamp,Group ID,Institution,S1 (Least Likely)\n\nYour first round of tests did not go well and your usual studying habits are not working,S1 (Most Likely) \n\nYour first round of tests did not go well and your usual studying habits are not working,S2 (Least Likely) \n\nYou want to get involved on campus but are unsure which activities to select,S2 (Most Likely) \n\nYou want to get involved on campus but are unsure which activities to select,S3 (Least Likely)\n\n You are having trouble finding engineering students with whom you relate and you are starting to feel socially isolated\n,S3 (MostLikely)\n\n You are having trouble finding engineering students with whom you relate and you are starting to feel socially isolated\n,...,S18 (Least Likely)\n\nYou no longer think engineering is the major for you but you are anxious about leaving engineering,S18 (Most Likely)\n\nYou no longer think engineering is the major for you but you are anxious about leaving engineering,S19 (Least Likely)\n\nYou have family or personal problems that are distracting you from school,S19 (Most Likely)\n\nYou have family or personal problems that are distracting you from school,"DQ1\n\nWhen you entered this institution, were you:",DQ2\n\nHow many semesters/quarters have you been an undergraduate engineering student at your current institution?\n,"DQ3\n\nDo you identify as neurodiverse (e.g., ADHD, anxiety, autism, bipolar disorder, dyslexia, etc.)?",DQ4\n\nHow would you describe your gender identity?,DQ5\n\nPlease indicate your race/ethnicity\n,DQ6\n\nAre you a first generation college student?\n
0,126,9/26/2024 14:18:12,no group,University Of Connecticut,Make and work with a study group with peers in...,Review your first round of tests once you get ...,Ask your friends about what organizations they...,Wait and see what opportunities you stumble ac...,Reach out to a mental health professional to t...,Join an engineering student organization,...,Stay in engineering in hopes that you will bec...,Talk to your academic advisor about wanting to...,Try to stay focused on your school work and ho...,Prioritize self care and take a break from sch...,A first-year college student directly from hig...,0-2,No,Man,Black or African American,"No, at least one of my parents has at least a ..."
1,127,9/27/2024 22:57:00,no group,Uconn,Wait until the next round of tests to see if y...,Start going to office hours regularly for each...,Search Instagram or other social media to find...,Ask your friends about what organizations they...,Reach out to a mental health professional to t...,Continue to focus on your studies and hope tha...,...,Vent to your friends or family about your disc...,Sign up for a non engineering class to explore...,Try to stay focused on your school work and ho...,Talk to your instructors about your situation ...,A first-year college student directly from hig...,0-2,"Yes, and I have an accommodation at my university",Man,Hispanic and/or Latino/a/x,"No, at least one of my parents has at least a ..."
2,128,9/28/2024 15:54:21,BRIDGE24,University of Connecticut,Visit the student success center to learn a ne...,Review your first round of tests once you get ...,Ask you advisor for recommendations based on y...,Attend campus organization fairs to learn abou...,Talk to family and friends from home to have f...,Continue to focus on your studies and hope tha...,...,Vent to your friends or family about your disc...,Talk to your academic advisor about wanting to...,Prioritize self care and take a break from sch...,Talk to your instructors about your situation ...,A first-year college student directly from hig...,0-2,No,Man,"African, Black or African American","No, at least one of my parents has at least a ..."
3,129,9/28/2024 19:30:02,BRIDGE24,University of Connecticut,Wait until the next round of tests to see if y...,Review your first round of tests once you get ...,Wait and see what opportunities you stumble ac...,Ask your friends about what organizatio

In [13]:
# === Use website order for each question, then encode to −1/0/+1 and save CSV ===
# Assumes you've already done:
# import pandas as pd
# df = pd.read_csv('/content/UConn_SJI_Data_.csv')

import re, numpy as np, pandas as pd
from difflib import get_close_matches

def _normalize_text(x):
    return re.sub(r"\s+", " ", str(x)).strip() if pd.notnull(x) else np.nan

# 1) Official order from the website (S1..S19), EXACT strings & order
choices_by_q = {
    1: [
        "Wait until the next round of tests to see if you’ll do better",
        "Review your first round of tests once you get them back to understand your mistakes",
        "Make and work with a study group with peers in your classes to better understand the material",
        "Start going to office hours regularly for each of your courses",
        "Visit the student success center to learn a new study method",
    ],
    2: [
        "Wait and see what opportunities you stumble across naturally",
        "Search Instagram or other social media to find organizations around campus that sound interesting to you",
        "Ask your friends about what organizations they have joined",
        "Ask you advisor for recommendations based on your major and/or interests",
        "Attend campus organization fairs to learn about organizations around campus",
    ],
    3: [
        "Continue to focus on your studies and hope that over time you will naturally encounter peers who share your interests",
        "Join a student organization outside of engineering",
        "Talk to family and friends from home to have familiar social interactions",
        "Join an engineering student organization",
        "Reach out to a mental health professional to talk about your feelings of isolation",
    ],
    4: [
        "Wait to think about purchasing books until you know if you need them for your classes",
        "Look online for a free or low cost pdf versions of your textbooks",
        "Ask a friend to borrow the books you need",
        "Talk to your instructor about your financial constraints to see if they can help you get access to the course materials at a cheaper rate",
        "Reach out to the financial aid office to see what financial support they can offer you to purchase required books",
    ],
    5: [
        "Continue your process of finding and applying to internships/co-ops in hopes that more applications will increase your chance of success",
        "Expand your search by looking on job websites liked Indeed",
        "Talk to your friends or family about advice for finding an internship or co-op",
        "Talk to your academic advisor about how to strengthen your resume",
        "Visit the career services office to get help with updating your resume",
    ],
    6: [
        "Wait for the instructor to respond to your request",
        "Find instructional videos or other open access educational resources to support your learning outside of class",
        "Make a study group with peers who are in the same class",
        "Reach out to the teaching assistant of the course to get help on the course material",
        "Get tutoring for the course from the student success center",
    ],
    7: [
        "Push through your challenging courses knowing the semester will eventually end",
        "Find instructional videos to watch online to support your learning outside of class",
        "Make and work with a study group of peers from your classes",
        "Ask your advisor for advice about time management",
        "Get tutoring for your courses from the student success center",
    ],
    8: [
        "Wait to pursue your outside interest when you have a less busy semester",
        "Carve out some time in your schedule to use towards your outside interest by deprioritizing some of your current work",
        "Ask a friend how they pursue interests outside of engineering",
        "Talk to your academic advisor about how to make time to pursue additional interests",
        "Reach out to career services to get advice about how to prioritize your time spent outside of engineering",
    ],
    9: [
        "Hope that this problem is temporary and will go away once the semester slows down",
        "Work on campus in a busy area to be surrounded by people",
        "Start a study group with friends to combine studying and socializing",
        "Talk to your advisor about the possibility of dropping a class to free up more time",
        "Reach out to a mental health professional to discuss your feelings of disconnection",
    ],
    10: [
        "Wait and see how you’re doing in the course once the deadline is closer",
        "Spend more time studying for the course to try to bring up your grade",
        "Ask a friend for help on the material you are struggling with to try to bring up your grade",
        "Talk to your academic advisor for advice about dropping the course",
        "Get tutoring for your courses from the student success center",
    ],
    11: [
        "Wait and see what opportunities show up in your email or around campus",
        "Search online for research opportunities",
        "Talk to peers who are doing undergraduate research about how they found their positions",
        "Ask your current instructors if any of them are interested in hiring undergraduate researchers",
        "Reach out to the undergraduate research office to get guidance",
    ],
    12: [
        "Ignore the treatment and see if it changes over time",
        "Avoid the people treating you unfairly",
        "Talk to a trusted friend or family member for assistance or advice",
        "Talk to one of your instructors about what you think is happening in their class and ask them to intervene",
        "Make an appointment to talk to a university employee such as an ombudsperson or a Title IX employee",
    ],
    13: [
        "Wait and see if the instructional quality of the course improves",
        "Find instructional videos or other open access educational resources to support your learning outside of class",
        "Make a study group with peers who are in the same class",
        "Reach out to another instructor of the same course to get help on the course material",
        "Get tutoring for the course from the student success center",
    ],
    14: [
        "Try not to compare yourself to your peers",
        "Spend more time on your assignments",
        "Talk with your peers about their study habits and methods for success",
        "Go to office hours regularly to get help on assignments from your instructors",
        "Reach out to a mental health professional to get help with managing your stress",
    ],
    15: [
        "Don’t apply for opportunities that require a recommendation letter",
        "Ask a prior or current employer for a letter of recommendation",
        "Ask one of your family members (or a friend of the family) for a recommendation letter",
        "Ask your advisor for a recommendation letter",
        "Reach out to the career services office to get advice",
    ],
    16: [
        "Keep working hard knowing that the semester will end eventually",
        "Make time for self care like going for a walk",
        "Reach out to a family member or friend to talk about your stress",
        "Talk to your instructors about your situation and ask for flexibility on your assignment deadlines",
        "Reach out to a mental health professional to talk about your high levels of stress",
    ],
    17: [
        "Stay active in the organization and see if the time commitment changes",
        "Talk to the leaders of the organization and request to take less responsibilities",
        "Talk to friends or family about balancing your organizational commitments and schoolwork",
        "Ask your academic advisor for advice about whether or not to stay active in the organization",
        "Talk to a student affairs professional (counselor/mentor/etc.) about how to manage all of your responsibilities effectively",
    ],
    18: [
        "Stay in engineering in hopes that you will become more comfortable",
        "Sign up for a non engineering class to explore your other interests",
        "Vent to your friends or family about your discomfort in engineering",
        "Talk to your academic advisor about wanting to leave engineering",
        "Reach out to career services to learn about career opportunities outside of engineering",
    ],
    19: [
        "Try to stay focused on your school work and hope that your personal problems will resolve on their own",
        "Prioritize self care and take a break from school work",
        "Talk to friends or family about your personal problems to help you process them",
        "Talk to your instructors about your situation and ask for flexibility on your assignment deadlines",
        "Reach out to a mental health professional to talk about personal or family problems",
    ],
}

# 2) Find S# (Most/Least) columns in your dataframe
SJI_HEADER_RE = re.compile(r"S\s*(\d+).*(Most|Least)\s*Likely", re.IGNORECASE)
def find_sji_columns(df):
    cols=[]
    for i,name in enumerate(df.columns):
        m=SJI_HEADER_RE.search(str(name))
        if m:
            cols.append({"q":int(m.group(1)), "sel":m.group(2).lower(), "name":name, "pos":i})
    # stable order: by question, Least then Most
    return sorted(cols, key=lambda d:(d["q"], 0 if d["sel"]=="least" else 1))

# 3) Build text→id maps per question using the official order
by_q_text2id = {q: { _normalize_text(t): i for i,t in enumerate(opts, start=1)}
                for q,opts in choices_by_q.items()}

def map_text_to_choice(q, raw):
    if pd.isna(raw): return np.nan
    return by_q_text2id.get(q, {}).get(_normalize_text(raw), np.nan)

# 4) Encode to −1/0/+1 (Most=+1, Least=−1; others=0)
sji_cols = find_sji_columns(df)
questions = sorted({c["q"] for c in sji_cols})
out_cols = [f"S{q:02d}_c{c}" for q in questions for c in range(1,6)]

encoded_rows=[]
issues=[]
RESPONDENT_COL = "Participant"  # optional id column

for ridx, row in df.iterrows():
    out = {c:0 for c in out_cols}
    if RESPONDENT_COL in df.columns:
        out["Participant"] = row[RESPONDENT_COL]
    for meta in sji_cols:
        q, sel, name = meta["q"], meta["sel"], meta["name"]
        cid = map_text_to_choice(q, row[name])
        if pd.isna(cid):
            issues.append({"row_index": int(ridx), "q": q, "sel": sel, "column": name,
                           "raw_text": _normalize_text(row[name]), "issue": "unmapped_text"})
            continue
        out[f"S{q:02d}_c{int(cid)}"] = 1 if sel=="most" else -1
    encoded_rows.append(out)

encoded_df = pd.DataFrame(encoded_rows)
ordered = (["Participant"] if "Participant" in encoded_df.columns else []) + out_cols
encoded_df = encoded_df.reindex(columns=ordered)
for c in out_cols: encoded_df[c] = encoded_df[c].astype("int8")

# 5) Save CSV (and optional download)
NUMERIC_OUT = "UConn_SJI_signed_onehot.csv"
encoded_df.to_csv(NUMERIC_OUT, index=False)
print("Saved:", NUMERIC_OUT, "shape:", encoded_df.shape)

from google.colab import files
files.download(NUMERIC_OUT)


Saved: UConn_SJI_signed_onehot.csv shape: (96, 96)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>